In [1]:
import sys
sys.path.append(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 7\LAB 4\timeseires')

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [3]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [4]:
def MLP():
    model = Sequential()
    model.add(Flatten(input_shape=(time_steps , num_features)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    return model

In [5]:
model1 = MLP()
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 504)               0         
                                                                 
 dense (Dense)               (None, 32)                16160     
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16,193
Trainable params: 16,193
Non-trainable params: 0
_________________________________________________________________


In [6]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [8]:
checkpoints = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 7\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 7'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [9]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [10]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =MLP()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [13]:
import os
path_dataset ='C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 7\\AEP_hourly_2'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [14]:
df_te.head(1)

,aep,Is_holiday1,Is_holiday2,Is_Weekend1,Is_Weekend2,sin_month,cos_month,sin_week,cos_week,sin_hour,...,sin_wintert,cos_wintert,sin_springt,cos_springt,sin_summert,cos_summert,sin_fallt,cos_fallt,sin_year_dayt,cos_year_dayt
0,0.518532,1.0,0.0,1.0,0.0,1.0,6.123234e-17,0.866025,-0.5,-0.707107,...,0.0,1.0,1.0,6.123234e-17,0.0,1.0,0.0,1.0,0.956235,0.2926


In [15]:
test_set[0]

array([ 5.18532385e-01,  1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  6.12323400e-17,  8.66025404e-01,
       -5.00000000e-01, -7.07106781e-01, -7.07106781e-01,  0.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  6.12323400e-17,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  1.00000000e+00,  9.56234827e-01,
        2.92600336e-01])

In [16]:
time_steps=2
num_features=21

In [18]:
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=0)

In [19]:
train_X[0]

array([[ 1.69100212e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         2.58819045e-01,  9.65925826e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.41395233e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         5.00000000e-01,  8.66025404e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02]])

In [20]:
train_y[0]

array([], dtype=float64)

In [21]:
time_steps =24
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=1,target_len=2)

In [22]:
train_X[0]

array([[ 1.69100212e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         2.58819045e-01,  9.65925826e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.41395233e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         5.00000000e-01,  8.66025404e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.26232372e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e

In [23]:
train_y[0]

array([1., 1.])

In [24]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 3.1332385540008545 sec


In [25]:
epochs = 60
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                    verbose = verbose)

Epoch 1/60
2653/2653 [==============================] - ETA: 0s - loss: 0.0497 - mae: 0.0497 - mape: 828.8320
Epoch 1: val_loss improved from inf to 0.02828, saving model to C:\Users\musta\Desktop\ML lab\lab 7\E1-cp-0001-loss0.03.h5
2653/2653 [==============================] - 25s 5ms/step - loss: 0.0497 - mae: 0.0497 - mape: 828.8320 - val_loss: 0.0283 - val_mae: 0.0283 - val_mape: 11.7744
Epoch 2/60
2638/2653 [============================>.] - ETA: 0s - loss: 0.0260 - mae: 0.0260 - mape: 519.4791
Epoch 2: val_loss improved from 0.02828 to 0.02492, saving model to C:\Users\musta\Desktop\ML lab\lab 7\E1-cp-0002-loss0.02.h5
2653/2653 [==============================] - 10s 4ms/step - loss: 0.0260 - mae: 0.0260 - mape: 516.6696 - val_loss: 0.0249 - val_mae: 0.0249 - val_mape: 11.3333
Epoch 3/60
2651/2653 [============================>.] - ETA: 0s - loss: 0.0231 - mae: 0.0231 - mape: 143.1700
Epoch 3: val_loss improved from 0.02492 to 0.02350, saving model to C:\Users\musta\Desktop\ML lab\

Epoch 24/60
2652/2653 [============================>.] - ETA: 0s - loss: 0.0132 - mae: 0.0132 - mape: 578.1100
Epoch 24: val_loss did not improve from 0.01172
2653/2653 [==============================] - 9s 3ms/step - loss: 0.0132 - mae: 0.0132 - mape: 577.9881 - val_loss: 0.0124 - val_mae: 0.0124 - val_mape: 5.6878
Epoch 25/60
2653/2653 [==============================] - ETA: 0s - loss: 0.0130 - mae: 0.0130 - mape: 382.4458
Epoch 25: val_loss did not improve from 0.01172
2653/2653 [==============================] - 10s 4ms/step - loss: 0.0130 - mae: 0.0130 - mape: 382.4458 - val_loss: 0.0119 - val_mae: 0.0119 - val_mape: 5.8859
Epoch 26/60
2642/2653 [============================>.] - ETA: 0s - loss: 0.0131 - mae: 0.0131 - mape: 293.6013
Epoch 26: val_loss did not improve from 0.01172
2653/2653 [==============================] - 12s 4ms/step - loss: 0.0131 - mae: 0.0131 - mape: 292.4477 - val_loss: 0.0126 - val_mae: 0.0126 - val_mape: 5.9925
Epoch 27/60
2646/2653 [=====================

In [26]:

model = load_model(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 7\E1-cp-0051-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 3ms/step
Mean Absolute Error (MAE): 166.55
Median Absolute Error (MedAE): 134.56
Mean Squared Error (MSE): 46593.16
Root Mean Squared Error (RMSE): 215.85
Mean Absolute Percentage Error (MAPE): 1.14 %
Median Absolute Percentage Error (MDAPE): 0.92 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


# Fine Tuning

In [27]:
checkpoints = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 7\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 7\E1-cp-0051-loss0.01.h5'
start_epoch= 52

In [28]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading C:\Users\musta\Desktop\ML lab\lab 7\E1-cp-0051-loss0.01.h5...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [29]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2648/2653 [============================>.] - ETA: 0s - loss: 0.0092 - mae: 0.0092 - mape: 211.2929
Epoch 1: val_loss improved from inf to 0.00912, saving model to C:\Users\musta\Desktop\ML lab\lab 7\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 14s 5ms/step - loss: 0.0092 - mae: 0.0092 - mape: 210.9340 - val_loss: 0.0091 - val_mae: 0.0091 - val_mape: 4.1544
Epoch 2/10
2641/2653 [============================>.] - ETA: 0s - loss: 0.0091 - mae: 0.0091 - mape: 225.8906
Epoch 2: val_loss did not improve from 0.00912
2653/2653 [==============================] - 12s 5ms/step - loss: 0.0091 - mae: 0.0091 - mape: 224.9192 - val_loss: 0.0092 - val_mae: 0.0092 - val_mape: 4.1883
Epoch 3/10
2643/2653 [============================>.] - ETA: 0s - loss: 0.0091 - mae: 0.0091 - mape: 229.8519
Epoch 3: val_loss did not improve from 0.00912
2653/2653 [==============================] - 11s 4ms/step - loss: 0.0091 - mae: 0.0091 - mape: 229.0323 - val_loss: 0.0092 - val_mae:

In [30]:

model = load_model(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 7\E2-cp-0009-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 2ms/step
Mean Absolute Error (MAE): 145.18
Median Absolute Error (MedAE): 114.85
Mean Squared Error (MSE): 36106.18
Root Mean Squared Error (RMSE): 190.02
Mean Absolute Percentage Error (MAPE): 0.99 %
Median Absolute Percentage Error (MDAPE): 0.8 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
